# NOTES:

- Waiting vs blocking  
--> blocking holds up everything (could be selective?)  
--> waiting for specific resources to reach inactive state (flags?)  

- Platemap vs positionmap
- Axes orientation

### TODO:
- tip touch
- pyVISA
- GUI
- try connect (error handling)
- tip touch

In [8]:
import ipywidgets as widgets
import serial as s
import yaml
import time
import sys
import numpy as np
import pandas as pd
import csv
from __future__ import division

In [ ]:
s1 = s.Serial(port='COM7', baudrate=9600)

In [ ]:
s1.close()

In [ ]:
# cmd = '/1Z2025000R'
# s1.write(cmd +'\r')
# time.sleep(0.2)
cmd = '/1QR'
s1.write(cmd +'\r')
time.sleep(0.2)
stri = s1.read(s1.inWaiting())
print stri.decode('utf8', 'ignore')

In [ ]:
cmd = '/1P500000R'
s1.write(cmd +'\r')
time.sleep(0.2)
stri = s1.read(s1.inWaiting())
print stri.decode('utf8', 'ignore')

In [ ]:
cmd = '/1V50000R'
s1.write(cmd +'\r')
time.sleep(0.2)
stri = s1.read(s1.inWaiting())
print stri.decode('utf8', 'ignore')

In [ ]:
s2 = s.Serial(port='COM6', baudrate=115200)

In [ ]:
s2.close()

# MOTOR
Lin Engineering 
1. Determine appropriate velocity_max = microsteps/sec
2. Determine motor limits
3. Determine conv = microsteps/mm
4. Determine orientation (P+; D-)

In [2]:
class Motor:
    def __init__(self, configFile, init=True):
        
#         f = open(configFile, 'r')
#         self.config = yaml.load(f)
        self.serial = s.Serial() # placeholder
        self.serial = s.Serial(port='COM7', baudrate=9600)
        
        self.prefix = '/1'
        self.terminator = '\r'
        self.conv = 2e6/99.24 # usteps/mm
        self.velocity_default = 1525320 # usteps/sec (75.6863784 mm/sec)
        self.velocity_limit = int(self.velocity_default/3) # usteps/sec
        self.ustep_max = 2025000 # usteps 100.48 mm
        self.ustep_min = 0

        if init:
            self.initialize()

    def initialize(self):
#         self.serial = s.Serial(**self.config['serial']) # open serial connection
        pass # TODO set moving current
        pass # TODO set holding current
        self.set_velocity(self.velocity_limit)  # set velocity
        self.home() # move motor to home
        
    def cmd(self, cmd_string, block=True):
        full_string = self.prefix + cmd_string + self.terminator
        self.serial.write(full_string)
        repr(full_string)
        time.sleep(0.15) # TODO: monitor for response?
        response = self.serial.read(self.serial.inWaiting()).decode('utf8', 'ignore')
        
        while block and self.is_busy():
            pass
           
        return response
    
#     # TEST cmd that prints full_string 
#     def cmd(self, cmd_string):
#         full_string = self.prefix + cmd_string + self.terminator
#         print full_string
    
    # velocity: (usteps/sec) 
    def set_velocity(self, velocity):
        if velocity > self.velocity_limit:
            velocity = self.velocity_limit
            print 'ERR: Desired velocity exceeds velocity_limit; velocity now set to velocity_limit'
            
        cmd_string = 'V{}R'.format(velocity)
        return self.cmd(cmd_string)
    
    def is_busy(self):
        cmd_string = '/1Q'
        time.sleep(0.05)
        response = self.cmd('/1Q', False)
        return response.rfind('`') == -1
            
    def halt(self):
        cmd_string = 'T'
        self.cmd(cmd_string)
    
    def home(self):
        cmd_string = 'Z{}R'.format(self.ustep_max)
        return self.cmd(cmd_string)
    
    # absolute
    def move(self, mm, block=True):
        ustep = int(self.conv*mm)
        if ustep > self.ustep_max:
            ustep = self.ustep_max
            print 'ERR: Desired move to {} mm exceeds max of {} mm; moving to max instead'.format(mm, self.ustep_max/self.conv)
        if ustep < self.ustep_min:
            ustep = self.ustep_min
            print 'ERR: Desired move to {} mm exceeds min of {} mm; moving to min instead'.format(mm, self.ustep_min/self.conv)
            
        cmd_string = 'A{}R'.format(ustep)
        
        return self.cmd(cmd_string, block)
        
    # TODO: Try P with negative numbers?
    def move_relative(self, mm):
        ustep = int(self.conv*mm)
        ustep_current = int(self.ustep_max/2)  # TODO: limit movement (+ and -)
        
        if mm >= 0:
            if (ustep_current + ustep) > self.ustep_max:
                ustep = self.ustep_max - ustep_current
                print 'ERR: Desired move of +{} mm exceeds max of {} mm; moving to max instead'.format(mm, self.ustep_max/self.conv)
            cmd_string = 'P{}R'.format(ustep)
        
        else:
            if (ustep_current + ustep) < self.ustep_min:
                ustep = self.ustep_min - ustep_current
                print 'ERR: Desired move of {} mm exceeds min of {} mm; moving to min instead'.format(mm, self.ustep_min/self.conv)
            ustep = -1*ustep
            cmd_string = 'D{}R'.format(ustep)
        
        return self.cmd(cmd_string)
    
    def exit(self):
        self.serial.close()


In [ ]:
m = Motor('blah')

In [ ]:
m.move(23)
# time.sleep(1)
# m.move(10)

In [ ]:
m.home()

In [ ]:
print m.move(32)
time.sleep(1)
print m.move(20)

In [ ]:
print m.cmd('P100000D100000P100000D100000P100000D100000P100000D100000R')
print m.cmd('/1?0')

In [ ]:
m.exit()

# ASI Controller
Applied Scientific Instrumentation

1. Set hall effect sensors to appropriate limits
2. Determine orientation (X+-, Y+-)

In [3]:
# TODO: Fix serial.read encoding
# TODO: better busy handling
class ASI_Controller:
    def __init__(self, configFile, init=True):
        f = open(configFile, 'r')
        self.config = yaml.load(f)
        self.serial = s.Serial() # placeholder
        self.plateMap = None

        if init:
            self.initialize()


    def initialize(self):
        self.serial = s.Serial(**self.config['serial']) # open serial connection

        self.cmd_xy('mc x+ y+')  # enable motor control for xy
        self.cmd_z('mc z+') # enable motor control for z

        print "Initializing stage..."
        self.move_xy(2000, -2000)  # move to switch limits (bottom right)
        self.r_xy(-0.5, 0.5)  # move from switch limits 0.5 mm

        raw_input('Place dropper above well A1 (press enter when done)')
        self.cmd_xy('here x y')  # establish current position as 0,0

        
    def cmd(self, inStr):
        inStr = inStr + self.config['terminator']
        self.serial.write(inStr)
        repr(inStr)
        time.sleep(0.05)
        response = self.serial.read(self.serial.inWaiting())
        return response

    def cmd_xy(self, inStr, block=True):
        inStr = '2h ' + inStr
        response = self.cmd(inStr)
        
        while block and self.is_busy_xy():
            time.sleep(0.05)
            pass
        
        return response
    
    def is_busy_xy(self):
        status = self.cmd('2h STATUS')[0]
        return status == 'B'

    
    def cmd_z(self, inStr, block=True):
        while block and self.is_busy_z():
            time.sleep(0.3)
        inStr = '1h ' + inStr
        return self.cmd(inStr)

    def is_busy_z(self):
        status = self.cmd('1h STATUS')[0]
        return status == 'B'

    def halt(self):
        self.halt_xy()
        self.halt_z()

    def halt_xy(self):
        self.cmd_xy('HALT', False)

    def halt_z(self):
        self.cmd_z('HALT', False)

    def r_xy(self, x_mm, y_mm):
        conv = self.config['conv']
        xStr = 'x=' + str(float(x_mm) * conv)
        yStr = 'y=' + str(float(y_mm) * conv)
        return self.cmd_xy(' '.join(['r', xStr, yStr]))

    def r_z(self, z_mm):
        conv = self.config['conv']
        zStr = 'z=' + str(float(z_mm) * conv)
        return self.cmd_z(' '.join(['r', zStr]))

    def move_xy(self, x_mm, y_mm):
        conv = self.config['conv']
        xStr = 'x=' + str(float(x_mm) * conv)
        yStr = 'y=' + str(float(y_mm) * conv)
        return self.cmd_xy(' '.join(['m', xStr, yStr]))

    def move_z(self, z_mm):
        conv = self.config['conv']
        zStr = 'z=' + str(float(z_mm) * conv)
        return self.cmd_z(' '.join(['m', zStr]))

    def exit(self):
        self.serial.close()





In [ ]:
a = ASI_Controller('components/controller.config')

In [ ]:
a.exit()

# Autosipper

In [4]:
# I: filepath of delimited file
# P: detect delimiter/header read file accordingly
# O: list of records (no header)
def read_delim(filepath):
    f = open(filepath, 'r')
    dialect = csv.Sniffer().sniff(f.read(1024))
    f.seek(0)
    hasHeader = csv.Sniffer().has_header(f.read(1024))
    f.seek(0)
    reader = csv.reader(f, dialect)

    if hasHeader:
        reader.next()

    ret = [line for line in reader]
    return ret

def read_delim_pd(filepath):
    f = open(filepath)
    has_header = None
    if csv.Sniffer().has_header(f.read(1024)):
        has_header = 0
    f.seek(0)
    return pd.read_csv(f, header=has_header, sep=None, engine='python')


def lookup(table, columns, values):
    temp_df = pd.DataFrame(data=[values], columns = columns, copy=False)
    return table.merge(temp_df, copy=False)

In [5]:
class Autosipper():
    def __init__(self):
        # add Motor
        # add ASI_Controller
        self.Z = Motor('blah')
        self.XY = ASI_Controller('components/controller.config')

        
        while True:
            fp = raw_input('Type in plate map file:')
            try:
                self.load_platemap(fp)  # load platemap
                break
            except IOError:
                print 'No file', fp

        raw_input('Place dropper above well A1 (press enter when done)')
        self.XY.cmd_xy('here x y')  # establish current position as 0,0
        
    def calibrate():
        pass
    
    def load_platemap(self, filepath):
        self.platemap = read_delim_pd(filepath)

    def go_to(self, columns, values):
        x1,y1,z1 = np.array(lookup(self.platemap, columns, values)[['x','y','z']])[0]

        self.Z.home() # move needle to travel height (blocking)
        self.XY.move_xy(x1,y1) # move stage (blocking)
        self.Z.move(z1) # move needle to bottom of well (blocking)
    
    def exit(self):
        self.XY.exit()
        self.Z.exit()
            

In [14]:
d = Autosipper()

Initializing stage...
Place dropper above well A1 (press enter when done)
Type in plate map file:platemaps/test.csv
Place dropper above well A1 (press enter when done)


In [ ]:
d.platemap

In [29]:
d.go_to(['n'],16)

In [ ]:
d.Z.move(10)
d.Z.move(30)

In [13]:
d.exit()

# PlatemapGen

## Calibration procedure:
Could also have dedicated reference position.   
PositionMap describes the whole deck layout. 
Define XYZ limits
Define travel height as min
1. Move needle tip to some reference point (e.g. center-bottom of well A01 of top plate). Record $X_r$ $Y_r$ $Z_r$.
2. Move

Define limits of hardware

In [6]:
# I:
# row_col_num: (ct, ct) 2-tuple of 
# row_col_spacing: (mm, mm)
# well_top_bottom_diameter: (mm, mm, mm)
# O: 
def generate_platemap(row_col_num, row_col_spacing, well_top_bottom_diameter):
    print 'n','s','r','c','name','x','y','z'
    for r in range(row_col_num[0]):
        for c in range(row_col_num[1]):
            n = c + (r)*row_col_num[0]
            s = ((r+1)%2)*(c + r*row_col_num[1]) + (r%2)*((r+1)*row_col_num[1] - (c+1))
            name = chr(64+r+1) + '{:02d}'.format(c+1)
            x = r*row_col_spacing[0]
            y = c*row_col_spacing[1]
            z = well_top_bottom_diameter[1]
            print n, s, r, c, name, x, y, z

row_col_num = 8,12
row_col_spacing = 9,9
well_top_bottom_diameter = 18,30,8
generate_platemap(row_col_num, row_col_spacing, well_top_bottom_diameter)

n s r c name x y z
0 0 0 0 A01 0 0 30
1 1 0 1 A02 0 9 30
2 2 0 2 A03 0 18 30
3 3 0 3 A04 0 27 30
4 4 0 4 A05 0 36 30
5 5 0 5 A06 0 45 30
6 6 0 6 A07 0 54 30
7 7 0 7 A08 0 63 30
8 8 0 8 A09 0 72 30
9 9 0 9 A10 0 81 30
10 10 0 10 A11 0 90 30
11 11 0 11 A12 0 99 30
8 23 1 0 B01 9 0 30
9 22 1 1 B02 9 9 30
10 21 1 2 B03 9 18 30
11 20 1 3 B04 9 27 30
12 19 1 4 B05 9 36 30
13 18 1 5 B06 9 45 30
14 17 1 6 B07 9 54 30
15 16 1 7 B08 9 63 30
16 15 1 8 B09 9 72 30
17 14 1 9 B10 9 81 30
18 13 1 10 B11 9 90 30
19 12 1 11 B12 9 99 30
16 24 2 0 C01 18 0 30
17 25 2 1 C02 18 9 30
18 26 2 2 C03 18 18 30
19 27 2 3 C04 18 27 30
20 28 2 4 C05 18 36 30
21 29 2 5 C06 18 45 30
22 30 2 6 C07 18 54 30
23 31 2 7 C08 18 63 30
24 32 2 8 C09 18 72 30
25 33 2 9 C10 18 81 30
26 34 2 10 C11 18 90 30
27 35 2 11 C12 18 99 30
24 47 3 0 D01 27 0 30
25 46 3 1 D02 27 9 30
26 45 3 2 D03 27 18 30
27 44 3 3 D04 27 27 30
28 43 3 4 D05 27 36 30
29 42 3 5 D06 27 45 30
30 41 3 6 D07 27 54 30
31 40 3 7 D08 27 63 30
32 39 3 8 D09 27 

# Communication: PyVISA

Install NI-VISA:
https://pyvisa.readthedocs.io/en/stable/getting_nivisa.html

In [ ]:
import visa

In [ ]:
rm = visa.ResourceManager()

In [ ]:
rm.list_resources()

In [ ]:
rm.list_resources_info()

# Documentation: Sphinx

http://thomas-cokelaer.info/tutorials/sphinx/docstring_python.html

# GUI: pywidgets.interact

http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [31]:
from IPython.display import display

In [170]:
x_minus = widgets.Button(
    description='<',
    disabled=False,
    button_style='',
    icon='check')

x_plus = widgets.Button(
    description='>',
    disabled=False,
    button_style='',
    icon='check')

y_minus = widgets.Button(
    description='^',
    disabled=False,
    button_style='', 
    icon='check', )

y_plus = widgets.Button(
    description='v',
    disabled=False,
    button_style='',
    icon='check')

xy_slider = widgets.VBox([widgets.FloatText(description='speed', width='50%',value=50),widgets.IntSlider(width=130, step=10)])
xy_cluster = widgets.VBox([ widgets.HBox([x_minus,x_plus]), widgets.HBox([y_minus, y_plus]) ])


z_minus = widgets.Button(
    description='^',
    disabled=False,
    button_style='', 
    icon='check')

z_plus = widgets.Button(
    description='v',
    disabled=False,
    button_style='',
    icon='check')

z_slider = widgets.VBox([widgets.FloatText(description='speed', width='50%',value=50),widgets.IntSlider(width=130, step=10)])
z_cluster = widgets.VBox([ z_minus, z_plus])


display(widgets.HBox([xy_cluster, xy_slider, z_cluster, z_slider]))